In [ ]:
import MDAnalysis as mda
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from MDAnalysis.analysis import align, rms

In [ ]:
PDB = "6cnj_prl20.pdb" #create a variable PDB for your structure file
XTC = "6cnj_prl20_25ns.xtc" #create a variable XTC for your trajectory file

In [ ]:
u = mda.Universe(PDB, XTC) #variable u compiling both structure and trajectory into one

print(u)
print(len(u.trajectory)) #just showing the size of your trajectory (kinda useless tbh)

In [ ]:
mobile = mda.Universe(PDB, XTC) #you are defining a variable as mobile just like the one you have for u
ref = mda.Universe(PDB, XTC) #same thing here

In [ ]:
mobile.trajectory[-1] #setting the mobile trajectory into one frame, your last one
ref.trajectory[0] #setting the reference trajectory as the first frame

mobile_ca = mobile.select_atoms('name CA')  #create a variable in which you select the alpha carbons
                                            #from the last frame (same terminology as used to select in VMD)
    
ref_ca = ref.select_atoms('name CA') #same thing for reference
rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition = False) #run an RMSD with the "non treated" frames

In [ ]:
aligner = align.AlignTraj(mobile, ref, select='protein and name CA', in_memory=True).run() #because you are not guaranteed
                                                                                           #that the trajectory frames are
                                                                                           #aligned, you may be getting some
                                                                                           #overall movement of the protein in
                                                                                           #solution. So, you align them.

In [ ]:
mobile.trajectory[-1] #then proceed to doing the same thing as before
ref.trajectory[0]

mobile_ca = mobile.select_atoms('backbone') #another way of looking at the overall movement of a protein is to call the backbone
                                            #instead of the alpha carbons only. It shouldn't give you too much of a difference.
ref_ca = ref.select_atoms('backbone')
rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition = False) #this RMSD should be smaller than the other one

In [ ]:
#An interesting trend that you may want to analyze is the motion of specific parts of your protein--like the c-loop. For that,
#you have to find out what are the atoms relevant for your c-loop and proceed with a somewhat similar structure as before.

mobile.trajectory[-1] #unsure if you need this, but I add just in case--could take out and see if it changes anything...
ref.trajectory[0]

mobile_ca = mobile.select_atoms('backbone and segid PROA and resid 197-204') #sometimes PROA, PROB, PROC etc stands for subunit
                                                                             #A,B,C etc. sometimes, it is only for part of it.
                                                                             #Make sure in VMD or Chimera that you are looking 
                                                                             #at the movement you want to look at.
ref_ca = ref.select_atoms('backbone and segid PROA and resid 197-204')
rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition = False)

In [ ]:
mobile_ca = mobile.select_atoms('backbone and segid PROG and resid 197-204')
ref_ca = ref.select_atoms('backbone and segid PROG and resid 197-204')
rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition = False)

In [ ]:
mobile_ca = mobile.select_atoms('backbone and segid PROE and resid 190-196')
ref_ca = ref.select_atoms('backbone and segid PROE and resid 190-196')
rms.rmsd(mobile_ca.positions, ref_ca.positions, superposition = False)

In [ ]:
#When plotting the data, you may want to decide on whether you will look at only the middle residue of your c-loop (which in
#theory should move the most), or the whole c-loop. Depending on what you want, you will use one of these sets of possibilities.

#C_loop_A = "backbone and segid PROA and resid 197-204"
#C_loop_C = "backbone and segid PROE and resid 190-196"
#C_loop_D = "backbone and segid PROG and resid 197-204"
C_loop_A = "backbone and segid PROA and resid 200"
C_loop_C = "backbone and segid PROE and resid 193"
C_loop_D = "backbone and segid PROG and resid 199"

R = rms.RMSD (mobile, ref, select='backbone',groupselections=[C_loop_A, C_loop_C, C_loop_D],ref_frame=0) #in variable R you are
                                                                                                         #storing RMSD values
                                                                                                         #of the backbone and
                                                                                                         #each c-loop selection
                                                                                                         #over time.

R.run() #because it is now a variable, you run it like this

In [ ]:
#To plot the R variable, you will envoke (or invoke idk) the commands dataframe from pandas (you may want to get yourself
#comfortable with this documentation in case you want to change the colors or the line strokes etc.

df = pd.DataFrame(R.rmsd, columns=['Frame', 'Time (ps)', 'Backbone','C_loop_A','C_loop_C','C_loop_D'])
df

In [ ]:
df['Time (ns)']=(df['Time (ps)']/1000)
df

In [ ]:
ax = df.plot(x='Time (ns)', 
             y=['C_loop_A','C_loop_C',"Backbone"],
             label=['C-loop ' r'$\alpha$/$\beta$','C-loop ' r'$\beta$/$\alpha$',"Backbone"],
             color=['#3FA039','#FF8837','blue'])

ax.set_ylabel(r'RMSD ($\AA$)')

plt.savefig('6cnj_prl20_RMSD.png')

RMSD Done

In [ ]:
#Doing the RMSF plot is very similar to the RMSD one, with changes in the specific commands.

c_alphas_A = mobile.select_atoms('protein and name CA and segid PROA')
RfA = rms.RMSF(c_alphas_A).run()

In [ ]:
#For some reason, I decided to use the library matplotlib to plot the RMSF. You may want to get used to this documentation
#as well.


plt.plot(c_alphas_A.resids, RfA.rmsf,color='#54529D')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.axvspan(197, 204, zorder=0, alpha=0.2, color='orange', label='C-loop')
plt.legend()

plt.savefig('6cnj_prl20_RMSF_A.png')

In [ ]:
c_alphas_E = mobile.select_atoms('protein and name CA and segid PROE')
RfB = rms.RMSF(c_alphas_E).run()

In [ ]:
plt.plot(c_alphas_E.resids, RfB.rmsf,color='#54529D')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.axvspan(190, 196, zorder=0, alpha=0.2, color='orange', label='C-loop')
plt.legend()

plt.savefig('6cnj_prl20_RMSF_C.png')

In [ ]:
c_alphas_G = mobile.select_atoms('protein and name CA and segid PROG')
RfC = rms.RMSF(c_alphas_G).run()

In [ ]:
plt.plot(c_alphas_G.resids, RfC.rmsf,color='#54529D')
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.axvspan(190, 196, zorder=0, alpha=0.2, color='orange', label='C-loop')
plt.legend();

RMSF Done!